In [1]:
### Install Google Cloud Vertex AI
# !pip install google-cloud-aiplatform
# # Install langchain
# ! pip install langchain
# # Install pandas
# ! pip install pandas
# # Install Python client for Google Search API
# ! pip install google-api-python-client
# # Install google-cloud-language
# ! pip install google-cloud-language
# # PDF loader
# ! pip install pypdf

In [3]:
# from google.colab import auth as google_auth
# google_auth.authenticate_user()

In [12]:
import vertexai
from vertexai.preview.language_models import CodeGenerationModel

WIDTH=600 # @param
PROJECT_ID="my-project-0004-346516" # @param
TEXT_MODEL="code-bison@001"#'text-bison@001' # @param 

In [13]:
def nl_to_sql(
    prefix_input: str,
    ) :
    """Codey Model Prediction."""
    vertexai.init(project=PROJECT_ID, location="us-central1")
    parameters = {
        "temperature": 0.5,
        "max_output_tokens": 256
    }
    model = CodeGenerationModel.from_pretrained(TEXT_MODEL)
    response = model.predict(
        prefix = prefix_input,
        **parameters
    )
    print(f"Question: {QUESTION}")
    print(f"Response from Model: {response.text}")

In [14]:
def build_prefix(
    question_input: str,
    ) :

    """Open prefix file and append question input from user."""
    file1 = open("prefix_file.txt")
    mystuff = file1.read()
    mystuff += "Question: {}\nSQL result: ".format(question_input)
    nl_to_sql(mystuff)

In [15]:
_prefix_file = 'prefix_file.txt'

In [16]:
QUESTION = "Top 3 user id with highest number of orders from order_line_items"

In [17]:
%%writefile {_prefix_file}
You are a database engineer that has profound SQL skills. You are using the following schema as reference to the database. Combine tables using JOIN if needed.

Schema =
[
    {
        "table_name": "surveys",
        "column_names": [
            primary_order_id,
            survey_type,
            survey_responded
         ]
     }, {
        "table_name": "order_line_items",
        "column_names": [
            source_system,
            primary_order_id,
            order_id,
            user_id,
            order_line_item_quantity,
            sale_price
         ]
     }, {
        "table_name": "orders",
        "column_names": [
            sent_to_warehouse,
            received_by_warehouse,
            split_shipment_order_type
         ]
     }, {
        "table_name": "monthly_subscription_order_retention",
        "column_names": [
            next_subscription_order_churn_type,
            sub_count
         ]
     },  {
         "table_name":"user_clusters",
         "column_names": [
            user_id,
            cluster_version,
            user_added_to_cluster_date,
            bd_id
         ]
      }, {
         "table_name":"order_delivery_performance",
         "column_names": [
            order_id,
            shipment_id,
            tracking_number,
            ship2d_busines
         ]
      }
]

Here's a simple training example for you to learn,
The question is "Tell me the most used coupon code for every coupon type."
And the sql query would be something like "SELECT coupon_type, coupon_code, SUM(coupon_redemptions) AS total_redemptions
FROM bark-dataengineering-poc.data_mart.promos
GROUP BY coupon_type, coupon_code
ORDER BY total_redemptions DESC;"

Give me the sql query result of the following question.


Overwriting prefix_file.txt


In [18]:
build_prefix(QUESTION)

Question: Top 3 user id with highest number of orders from order_line_items
Response from Model: SELECT user_id, COUNT(1) AS order_count
FROM order_line_items
GROUP BY user_id
ORDER BY order_count DESC
LIMIT 3;
